# CSV animation
RFF I, RFF II, Mezei II

In [ ]:
import numpy as np
from qutip import *
import pandas as pd
import os
import subprocess
from IPython import display
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:

def create_video(path, name, fps=30):
    command = f'ffmpeg -framerate {fps} -i {path}/frame_%d.png -c:v libx264 -pix_fmt yuv420p {name}.mp4'
    subprocess.run(command, shell=True, check=True)

def create_gif(path, name):
    command = f'ffmpeg -framerate 30 -i {path}/frame_%d.png {name}.gif'
    subprocess.run(command, shell=True, check=True)

def display_gif(fn):
    return display.HTML('<img src="{}">'.format(fn))

def display_video(video_path):
    return display.Video(video_path)

In [ ]:
def csvtovector(file_name, skiprows=0):
    # Load the csv file
    df_tmp = pd.read_csv(file_name, skiprows=skiprows)
    df_tmp.columns = df_tmp.columns.str.strip()

    # select columns and copy
    df = df_tmp[['Sx', 'Sy', 'Sz']].copy()

    # Convert all the dataframe values to numeric, invalid parsing will be set as NaN
    df = df.apply(pd.to_numeric, errors='coerce')

    # Drop the rows where at least one element is missing
    df = df.dropna()

    # convert columns to a list of lists
    vectors = df[['Sx', 'Sy', 'Sz']].values.tolist()
    return vectors

In [ ]:
def vectortoframes(vectors, save_path):
    if not os.path.exists(save_path):          # Funtion 'folder' is here
        os.makedirs(save_path)
    cmap = plt.get_cmap('seismic')
    num_of_points = len(vectors)
    colors = [cmap(i/num_of_points) for i in range(num_of_points)]

    b = Bloch()

    b.zlabel = ['|↑⟩', '|↓⟩']
    b.point_marker = ['o']  # Set all point markers to circles
    b.point_size = [20]
    b.vector_color = ['green']
    b.point_color = colors

    n = 0
    for i in range(len(vectors)):
        b.add_vectors(vectors[i])
        b.save(f'{save_path}/frame_{n}.png', format='png')
        b.add_points(vectors[i])
        b.save(f'{save_path}/frame_{n+1}.png', format='png')
        b.clear()
        for j in range(i+1):
            b.add_points(vectors[j])
        n += 2

In [ ]:
vectors = csvtovector('csv_data/RF-Flipper_2.csv')
vectortoframes(vectors, 'rff2_tmp')

In [ ]:
vectors1 = csvtovector('csv_data/MEZEI-Flipper_2.csv')
vectortoframes(vectors1, 'mf2_tmp')

In [ ]:
vectors2 = csvtovector('csv_data/RF-Flipper_1.csv')
vectortoframes(vectors2, 'rff1_tmp')

In [ ]:
create_video('rff1_tmp', 'rff1', 120)

In [ ]:
create_video('rff2_tmp', 'rff2', 120)

In [ ]:
create_video('mf2_tmp', 'mf2', 60)